In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import random

import warnings
warnings.filterwarnings("ignore")

data_path = '/content/drive/MyDrive/SolarGen/'

## Baseline

In [ ]:
df_train = pd.read_csv(data_path + './data/train/train.csv')
submission = pd.read_csv(data_path + './data/sample_submission.csv')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import (
    MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer, 
    MaxAbsScaler, PowerTransformer
)

class ScalingFactor:
    def __init__(self, scaler, features):
        self.scaler = scaler
        self.features = features

def scaling(data, scaler, features, is_train=True):

    temp = data.copy()

    target_cols = features
    temp = temp[target_cols]
    if is_train: 
        scaler.fit(temp)
    scaled = scaler.transform(temp)

    data[target_cols] = scaled

    return data     

def preprocess_data(data, scalers, is_train=True):
    
    temp = data.copy()

    for factor in scale_factors:
        temp = scaling(temp, factor.scaler, factor.features, is_train)

    temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]

    if is_train==True:          
    
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill')
        temp = temp.dropna()
        
        return temp.iloc[:-96]

    elif is_train==False:
        
        temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]
                              
        return temp.iloc[-48:, :]

scale_factors = []
# scale_factors.append(ScalingFactor(PowerTransformer(), ['DHI', 'DNI']))
# scale_factors.append(ScalingFactor(QuantileTransformer(output_distribution='normal'), ['DNI']))
# scale_factors.append(ScalingFactor(StandardScaler(), ['RH', 'T']))
# scale_factors.append(ScalingFactor(MinMaxScaler(), ['DHI', 'DNI', 'WS', 'RH', 'T']))

train = preprocess_data(df_train, scale_factors)
train.iloc[:, 1:].plot(subplots=True, layout=(1,8), figsize=(30,4))
train.iloc[:, 1:].hist(bins = 100, layout=(1,8), figsize=(30,4))

In [ ]:
df_test = []

for i in range(81):
    file_path = data_path + './data/test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, scale_factors, is_train=False)
    df_test.append(temp)

X_test = pd.concat(df_test)

X_test.iloc[:, 1:].plot(subplots=True, layout=(1,6), figsize=(30,4))
X_test.iloc[:, 1:].hist(bins = 100, layout=(1,6), figsize=(30,4))

In [ ]:
from sklearn.model_selection import train_test_split
X_train_1, X_valid_1, Y_train_1, Y_valid_1 = train_test_split(df_train.iloc[:, :-2], df_train.iloc[:, -2], test_size=0.3, random_state=0)
X_train_2, X_valid_2, Y_train_2, Y_valid_2 = train_test_split(df_train.iloc[:, :-2], df_train.iloc[:, -1], test_size=0.3, random_state=0)

In [ ]:
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
from lightgbm import LGBMRegressor

# Get the model and the predictions in (a) - (b)
def LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    # (a) Modeling  
    model = LGBMRegressor(objective='quantile', alpha=q,
                         n_estimators=10000, bagging_fraction=0.7, learning_rate=0.027, subsample=0.7)                   
                         
                         
    model.fit(X_train, Y_train, eval_metric = ['quantile'], 
          eval_set=[(X_valid, Y_valid)], early_stopping_rounds=300, verbose=500)

    # (b) Predictions
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

In [ ]:
# Target 예측
import sys
from io import StringIO

def get_best_quantiles(data):

    best_indexes = []
    best_quantiles = {}

    lines = data.splitlines()

    for i, line in enumerate(lines):
        if "best" in line:
            best_indexes.append(i+1)

    for i in range(9):
        key = (i+1) * 0.1
        val = float(lines[best_indexes[i]].split(":")[-1].strip())
        best_quantiles[key] = val

    return best_quantiles

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):

    LGBM_models=[]
    LGBM_actual_pred = pd.DataFrame()
    
    old_stdout = sys.stdout
    sys.stdout = my_stdout = StringIO()
    for q in quantiles:
        print(q)
        pred , model = LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test)
        LGBM_models.append(model)
        LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1)
    
    sys.stdout = old_stdout
    results_out = my_stdout.getvalue()
    LGBM_losses = get_best_quantiles(results_out)

    LGBM_actual_pred.columns=quantiles
    
    return LGBM_models, LGBM_actual_pred, LGBM_losses

In [ ]:
# Target1
models_1, results_1, loss_1 = train_data(X_train_1, Y_train_1, X_valid_1, Y_valid_1, X_test)
results_1.sort_index()[:48]

In [ ]:
for key, val in loss_1.items():
    print("[%.1f]"%key + "\t%f"%val)
print("[avg]\t%.6f"%np.average(list(loss_1.values())))

In [ ]:
# Target2
models_2, results_2, loss_2 = train_data(X_train_2, Y_train_2, X_valid_2, Y_valid_2, X_test)
results_2.sort_index()[:48]

In [ ]:
for key, val in loss_2.items():
    print("[%.1f]"%key + "\t%f"%val)
print("[avg]\t%.6f"%np.average(list(loss_2.values())))

In [ ]:
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.values
submission

In [ ]:
submission.iloc[:48]

In [ ]:
# submission.to_csv(data_path + './data/submission.csv', index=False)